# The Battle of the Neighborhoods 
### Capstone Project by IBM/Coursera

First we will install and import all required libraries.

In [2]:
!conda install -c conda-forge geopy --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



## Package Plan ##

  environment location: /home/jupyterlab/conda/envs/python

  added / updated specs: 
    - geopy


The following packages will be downloaded:

    package                    |            build
    ---------------------------|-----------------
    geopy-1.20.0               |             py_0          57 KB  conda-forge
    certifi-2019.9.11          |           py36_0         147 KB  conda-forge
    geographiclib-1.49         |             py_0          32 KB  conda-forge
    ------------------------------------------------------------
                                           Total:         237 KB

The following NEW packages will be INSTALLED:

    geographiclib: 1.49-py_0        conda-forge
    geopy:         1.20.0-py_0      conda-forge

The following pac

In [3]:
!conda install -c conda-forge folium=0.5.0 --yes

Solving environment: done


==> WARNING: A newer version of conda exists. <==
  current version: 4.5.11
  latest version: 4.7.12

Please update conda by running

    $ conda update -n base -c defaults conda



# All requested packages already installed.



In [3]:
import numpy as np # library to handle data in a vectorized manner

import pandas as pd # library for data analsysis
pd.set_option('display.max_columns', None)
pd.set_option('display.max_rows', None)

import json # library to handle JSON files

#!conda install -c conda-forge geopy --yes # uncomment this line if you haven't completed the Foursquare API lab
from geopy.geocoders import Nominatim # convert an address into latitude and longitude values

import requests # library to handle requests
from pandas.io.json import json_normalize # tranform JSON file into a pandas dataframe

# Matplotlib and associated plotting modules
import matplotlib.cm as cm
import matplotlib.colors as colors

# map rendering library
import folium

## Table of contents
* [Introduction](#introduction)
* [Data](#data)
* [Methodology](#methodology)
* [Analysis](#analysis)
* [Results and Discussion](#results)
* [Conclusion](#conclusion)

## 1. Introduction <a name="introduction"></a>

### Business Problem
There are over a hundred restaurants in Chicago. Most of them are American restaurants and cafes. To open a new cafe, we need to find the best location in the city. The optimal location would be one less crowded with other restaurants especially coffee shops and other cafes. Also the site should not be far from the city center. Once the above two conditions are satisfied we will look for locations closer to the city center. This report is meant to help stakeholders who are interested in opening a cafe in Chicago. We will use data science methodologies to find the most promising sites and mention their advantages. Stakeholders can choose the best location according to their preferences.      
This project aims to find the best land site to start an cafe in Chicago. Based on definition of our problem, factors that will influence our decision are:
* number of existing restaurants in the neighborhood
* number of and distance to cafes in the neighborhood, if any
* distance of neighborhood from city center


## 2. Data <a name="data"></a>

 
Data is collected from kaggle website. Chicago neighborhoods data is a json file that contains the longitude and latitudes of neighborhoods. The link to the data is :
https://www.kaggle.com/afernandezcan/chicago-neighborhoods-2012


#### Load and explore the data

Data is downloaded as a zip file from kaggle site and the json file is extracted. After that, it is read into a *pandas* dataframe.

In [4]:
import pandas as pd
import json

with open('Neighborhoods_2012_polygons.json') as f:
   data = json.load(f)

data

{'type': 'FeatureCollection',
 'crs': {'type': 'name',
  'properties': {'name': 'urn:ogc:def:crs:OGC:1.3:CRS84'}},
 'features': [{'type': 'Feature',
   'properties': {'PRI_NEIGH': 'Grand Boulevard',
    'SEC_NEIGH': 'BRONZEVILLE',
    'SHAPE_AREA': 48492503.1554,
    'SHAPE_LEN': 28196.837157},
   'geometry': {'type': 'Polygon',
    'coordinates': [[[-87.60670812560363, 41.81681377137387],
      [-87.60670480953505, 41.81657908583579],
      [-87.60670022648407, 41.816338713552454],
      [-87.60669581538588, 41.816099357727296],
      [-87.60668982110376, 41.8158118024656],
      [-87.60668357216157, 41.81556631526606],
      [-87.60667660553894, 41.815299912163404],
      [-87.6066796364493, 41.814994168113515],
      [-87.60668235893172, 41.81471953500853],
      [-87.60667153481008, 41.8142816453241],
      [-87.60666414094068, 41.81399460252956],
      [-87.60665643548599, 41.81366052091469],
      [-87.6066508943903, 41.81342058153228],
      [-87.60664694986733, 41.8131732001668

In [5]:
neighborhoods_data = data['features']

In [6]:
neighborhoods_data[0]

{'type': 'Feature',
 'properties': {'PRI_NEIGH': 'Grand Boulevard',
  'SEC_NEIGH': 'BRONZEVILLE',
  'SHAPE_AREA': 48492503.1554,
  'SHAPE_LEN': 28196.837157},
 'geometry': {'type': 'Polygon',
  'coordinates': [[[-87.60670812560363, 41.81681377137387],
    [-87.60670480953505, 41.81657908583579],
    [-87.60670022648407, 41.816338713552454],
    [-87.60669581538588, 41.816099357727296],
    [-87.60668982110376, 41.8158118024656],
    [-87.60668357216157, 41.81556631526606],
    [-87.60667660553894, 41.815299912163404],
    [-87.6066796364493, 41.814994168113515],
    [-87.60668235893172, 41.81471953500853],
    [-87.60667153481008, 41.8142816453241],
    [-87.60666414094068, 41.81399460252956],
    [-87.60665643548599, 41.81366052091469],
    [-87.6066508943903, 41.81342058153228],
    [-87.60664694986733, 41.81317320016689],
    [-87.60664346744446, 41.81295477653956],
    [-87.60663536449937, 41.812655873932044],
    [-87.60662867423531, 41.81240396220561],
    [-87.60662516618756, 41

#### Tranform the data into a *pandas* dataframe

In [7]:
# define the dataframe columns
column_names = ['Neighborhood', 'Latitude', 'Longitude'] 

# instantiate the dataframe
neighborhoods = pd.DataFrame(columns=column_names)
neighborhoods

,Neighborhood,Latitude,Longitude


#### Finding latitude and longitude of Chicago

In [8]:
address = 'Chicago'

geolocator = Nominatim(user_agent="ch_explorer")
location = geolocator.geocode(address)
latitude = location.latitude
longitude = location.longitude
print('The geograpical coordinate of Chicago are {}, {}.'.format(latitude, longitude))
chicago=[latitude, longitude]

The geograpical coordinate of Chicago are 41.8755616, -87.6244212.


Defining 3 functions:  
    1. lonlat_to_xy --- converts longitude and latitude to coordinates  
    2. xy_to_lonlat --- converts coordinates to longitude and latitude  
    3. calc_xy_distance --- calculates distance between set of coordinates  

In [9]:
!pip install shapely
!pip install pyproj
import pyproj

import math

def lonlat_to_xy(lon, lat):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    xy = pyproj.transform(proj_latlon, proj_xy, lon, lat)
    return xy[0], xy[1]

def xy_to_lonlat(x, y):
    proj_latlon = pyproj.Proj(proj='latlong',datum='WGS84')
    proj_xy = pyproj.Proj(proj="utm", zone=33, datum='WGS84')
    lonlat = pyproj.transform(proj_xy, proj_latlon, x, y)
    return lonlat[0], lonlat[1]

def calc_xy_distance(x1, y1, x2, y2):
    dx = x2 - x1
    dy = y2 - y1
    return math.sqrt(dx*dx + dy*dy)

Coordinates of Chicago center -- X, Y.

In [10]:
X,Y=lonlat_to_xy(41.8755616, -87.6244212) 

Building the new dataframe with longitude, latitude, coordinates and distance from center of each neighborhood.

In [11]:
for data in neighborhoods_data:
    distance_from_center=[]
    neighborhood_name = data['properties']['PRI_NEIGH']
        
    neighborhood_latlon = data['geometry']['coordinates']
    neighborhood_lat = neighborhood_latlon[0][0][1]
    neighborhood_lon = neighborhood_latlon[0][0][0]
    
    x,y=lonlat_to_xy(neighborhood_lon, neighborhood_lat)
    distance_from_center = calc_xy_distance(X, Y, x, y)
    
    neighborhoods = neighborhoods.append({'Neighborhood': neighborhood_name,
                                          'Latitude': neighborhood_lat,
                                          'Longitude': neighborhood_lon,
                                          'X': x,
                                          'Y': y,
                                        'Distance': distance_from_center}, ignore_index=True)

TypeError: unsupported operand type(s) for -: 'list' and 'float'

In [ ]:
neighborhoods.head()

### Foursquare
Now that we have our location candidates, let's use Foursquare API to get info on restaurants in each neighborhood.

We're interested in venues in 'food' category. We will include in out list venues that have 'restaurant', 'cafe', 'cafeteria', 'bakery', 'coffee shop'  in category name, since they can all be considered competitors and we'll make sure to detect and include all the subcategories of specific 'cafes' category, as we need info on cafes in the neighborhood.

Next, we are going to start utilizing the Foursquare API to explore the neighborhoods and segment them.

In [ ]:
CLIENT_ID = 'YWI34KNLNTT5G30DTQRXRJOCL5RYCD1WJWA5O53AOA2TWFBU' # your Foursquare ID
CLIENT_SECRET = '4USNPFE2UQNMHSCIW2I5XOQ33AFCSRXB2OP1YLZXJRTGN3IX' # your Foursquare Secret
VERSION = '20180605' # Foursquare API version

print('Your credentails:')
print('CLIENT_ID: ' + CLIENT_ID)
print('CLIENT_SECRET:' + CLIENT_SECRET)

In [12]:
# Category IDs corresponding to cafes were taken from Foursquare web site (https://developer.foursquare.com/docs/resources/categories):

food_category = '4d4b7105d754a06374d81259' # 'Root' category for all food-related venues

cafe_categories = ['4bf58dd8d48988d1e0931735','4bf58dd8d48988d16d941735']

def is_restaurant(categories, specific_filter=None):
    restaurant_words = ['restaurant', 'diner', 'taverna', 'steakhouse', 'café', 'cafe', 'cafeteria', 'bakery', 'coffee shop']
    restaurant = False
    specific = False
    for c in categories:
        category_name = c[0].lower()
        category_id = c[1]
        for r in restaurant_words:
            if r in category_name:
                restaurant = True
        if 'fast food' in category_name:
            restaurant = False
        if not(specific_filter is None) and (category_id in specific_filter):
            specific = True
            restaurant = True
    return restaurant, specific

def get_categories(categories):
    return [(cat['name'], cat['id']) for cat in categories]

def format_address(location):
    address = ', '.join(location['formattedAddress'])
    address = address.replace(', Deutschland', '')
    address = address.replace(', Germany', '')
    return address

def get_venues_near_location(lat, lon, category, CLIENT_ID, CLIENT_SECRET, radius=500, limit=100):
    version = '20180724'
    url = 'https://api.foursquare.com/v2/venues/explore?client_id={}&client_secret={}&v={}&ll={},{}&categoryId={}&radius={}&limit={}'.format(
        CLIENT_ID, CLIENT_SECRET, version, lat, lon, category, radius, limit)
    try:
        results = requests.get(url).json()['response']['groups'][0]['items']
        venues = [(item['venue']['id'],
                   item['venue']['name'],
                   get_categories(item['venue']['categories']),
                   (item['venue']['location']['lat'], item['venue']['location']['lng']),
                   format_address(item['venue']['location']),
                   item['venue']['location']['distance']) for item in results]        
    except:
        venues = []
    return venues

In [13]:
# Let's now go over our neighborhood locations and get nearby restaurants; we'll also maintain a dictionary of all found restaurants and all found cafes

import pickle

def get_restaurants(lats, lons):
    restaurants = {}
    cafe_places = {}
    location_restaurants = []

    print('Obtaining venues around candidate locations:', end='')
    for lat, lon in zip(lats, lons):
        # Using radius=350 to meke sure we have overlaps/full coverage so we don't miss any restaurant (we're using dictionaries to remove any duplicates resulting from area overlaps)
        venues = get_venues_near_location(lat, lon, food_category, CLIENT_ID, CLIENT_SECRET, radius=1000, limit=100)
        area_restaurants = []
        for venue in venues:
            venue_id = venue[0]
            venue_name = venue[1]
            venue_categories = venue[2]
            venue_latlon = venue[3]
            venue_address = venue[4]
            venue_distance = venue[5]
            is_res, is_cafe = is_restaurant(venue_categories, specific_filter=cafe_categories)
            if is_res:
                x, y = lonlat_to_xy(venue_latlon[1], venue_latlon[0])
                restaurant = (venue_id, venue_name, venue_latlon[0], venue_latlon[1], venue_address, venue_distance, is_cafe, x, y)
                if venue_distance<=3000:
                    area_restaurants.append(restaurant)
                restaurants[venue_id] = restaurant
                if is_cafe:
                    cafe_places[venue_id] = restaurant
        location_restaurants.append(area_restaurants)
        print(' .', end='')
    print(' done.')
    return restaurants, cafe_places, location_restaurants

# Try to load from local file system in case we did this before
restaurants = {}
cafe_places = {}
location_restaurants = []
loaded = False
try:
    with open('restaurants_350.pkl', 'rb') as f:
        restaurants = pickle.load(f)
    with open('cafe_places_350.pkl', 'rb') as f:
        cafe_places = pickle.load(f)
    with open('location_restaurants_350.pkl', 'rb') as f:
        location_restaurants = pickle.load(f)
    print('Restaurant data loaded.')
    loaded = True
except:
    pass

# If load failed use the Foursquare API to get the data
if not loaded:
    restaurants, cafe_places, location_restaurants = get_restaurants(neighborhoods.Latitude, neighborhoods.Longitude)
    
    # Let's persists this in local file system
    with open('restaurants_350.pkl', 'wb') as f:
        pickle.dump(restaurants, f)
    with open('cafe_places_350.pkl', 'wb') as f:
        pickle.dump(cafe_places, f)
    with open('location_restaurants_350.pkl', 'wb') as f:
        pickle.dump(location_restaurants, f)
        

Restaurant data loaded.


In [14]:
import numpy as np

print('Total number of restaurants:', len(restaurants))
print('Total number of cafes:', len(cafe_places))
print('Percentage of cafes: {:.2f}%'.format(len(cafe_places) / len(restaurants) * 100))
print('Average number of restaurants in neighborhood:', np.array([len(r) for r in location_restaurants]).mean())

Total number of restaurants: 750
Total number of cafes: 47
Percentage of cafes: 6.27%
Average number of restaurants in neighborhood: 22.8


In [15]:
print('List of all restaurants')
print('-----------------------')
for r in list(restaurants.values())[:10]:
    print(r)
print('...')
print('Total:', len(restaurants))

List of all restaurants
-----------------------
('4caf82eceb65b1f78c4d64cd', "Norman's Bistro", 41.81679481240921, -87.60180942790595, '1001 E 43rd St (at Ellis), Chicago, IL 60653, United States', 801, False, -5389668.372163451, 11522317.383352159)
('5754a10f498e66ad7d1ac738', 'Two Fish Crab Shack', 41.80938569255865, -87.60970578074011, '641 E 47th St, Chicago, IL 60653, United States', 863, False, -5390501.1035615895, 11523601.489143636)
('4e9a10177ee539c8570a838b', 'De Rice', 41.8095831686301, -87.60296336535757, '920 E 47th St (at S Drexel Ave), Chicago, IL 60653, United States', 862, False, -5390729.0235400265, 11522818.582572583)
('565a07f4498eb5055bb914e9', 'Café on the Grove', 41.816292833754474, -87.60647578271298, '4317 S Cottage Grove, Chicago, IL, United States', 61, True, -5389566.6805222705, 11522877.808733989)
('53a84caa498ec4faa093c335', 'Black Rose Pastries', 41.81851564135961, -87.6052391302644, '47th (Halsted), Chicago, IL 60653, United States', 225, False, -5389273

In [16]:
print('List of cafes')
print('---------------------------')
for r in list(cafe_places.values())[:10]:
    print(r)
print('...')
print('Total:', len(cafe_places))

List of cafes
---------------------------
('565a07f4498eb5055bb914e9', 'Café on the Grove', 41.816292833754474, -87.60647578271298, '4317 S Cottage Grove, Chicago, IL, United States', 61, True, -5389566.6805222705, 11522877.808733989)
('553977b7498eb9779dcb0592', 'Goddess and The Baker', 41.8813047949221, -87.62611682811053, '33 S Wabash Ave, Chicago, IL 60603, United States', 781, True, -5378862.954801424, 11521800.852922337)
('57b485e1498e8f5a79049391', 'Revival Cafe-Bar RCB', 41.879534074819816, -87.63063625141754, '125 S Clark St (W Adams St), Chicago, IL 60603, United States', 627, True, -5378961.157711256, 11522408.121066185)
('56f80be2cd104c9189b44b4c', 'Art Institute Museum Cafe', 41.87961122566739, -87.62184954164597, '111 S Michigan Ave (LL), Chicago, IL 60603, United States', 753, True, -5379285.089156254, 11521399.502669042)
('4d408eefaae1b1f7e6fd87f5', 'Caffè Moderno', 41.87988017397209, -87.62211140514324, 'Art Institute - The Modern Wing (159 E. Monroe), Chicago, IL 6060

In [17]:
print('restaurants around location')
print('---------------------------')
for i in range(0, 5):
    rs = location_restaurants[i][:8]
    names = ', '.join([r[1] for r in rs])
    print('restaurants around location {}: {}'.format(i+1, names))

restaurants around location
---------------------------
restaurants around location 1: Norman's Bistro, Two Fish Crab Shack, De Rice, Café on the Grove, Black Rose Pastries, Golden Fish and Chicken, Taqueria Varitas, Sharks Fish and Chicken
restaurants around location 2: Cafecito, Meli Cafe, UMAI Japanese Kitchen & Sushi, Mercat a la Planxa, Himalayan Restaurant, Sofi Restaurant, S.R.O. - Standing Room Only, Spanglish
restaurants around location 3: Uncle Mike's Place, Eden, The Loyalist, Big Delicious Planet Catering & Canteen, Smyth, Gyu-Kaku Japanese BBQ, Output, Elske
restaurants around location 4: Floriole Cafe & Bakery, Tarantino's, Range, Chipotle Mexican Grill, Sai Cafe, Café Ba-Ba-Reeba!, Athenian Room, White Oak Tavern & Inn
restaurants around location 5: Jerk Taco Stand, Royal Chicken, Maxine's Jamaica Cuisine, Joye Chinese


In [54]:
map_chicago = folium.Map(location=chicago, zoom_start=13)
folium.Marker(chicago, popup='Chicago').add_to(map_chicago)
for res in restaurants.values():
    lat = res[2]; lon = res[3]
    is_cafe = res[6]
    color = 'red' if is_cafe else 'blue'
    folium.CircleMarker([lat, lon], radius=3, color=color, fill=True, fill_color=color, fill_opacity=1).add_to(map_chicago)
map_chicago

Now we have all the restaurants in area within few kilometers from Chicago center, and we know which ones are cafes! We also know which restaurants exactly are in vicinity of every neighborhood candidate center.

This concludes the data gathering phase - we're now ready to use this data for analysis to produce the report on optimal locations for a new cafe!

## Methodology <a name="methodology"></a>

In this project we will direct our efforts on detecting areas of Chicago that have low restaurant density, particularly those with low number of cafes. We will limit our analysis to area ~6km around city center.

In first step we have collected the required **data: location and type (category) of every restaurant within 6km from Chicago center**. We have also **identified cafes** (according to Foursquare categorization).

Second step in our analysis will be calculation and exploration of '**restaurant density**' across different areas of Chicago - we will use **heatmaps** to identify a few promising areas close to center with low number of restaurants in general (*and* no cafes in vicinity) and focus our attention on those areas.

In third and final step we will take into consideration locations with **no more than one restaurant in radius of 500 meters**, and we want locations **without cafes in radius of 400 meters**. We will present map of all such locations but also create clusters (using **k-means clustering**) of those locations to identify general zones / neighborhoods / addresses which should be a starting point for final 'street level' exploration and search for optimal venue location by stakeholders.

## Analysis <a name="analysis"></a>

Let's perform some basic explanatory data analysis and derive some additional info from our raw data. First let's count the *number of restaurants in every area candidate**:

In [55]:
location_restaurants_count = [len(res) for res in location_restaurants]

neighborhoods['Restaurants in area'] = location_restaurants_count

print('Average number of restaurants in every area with radius=300m:', np.array(location_restaurants_count).mean())

neighborhoods.head(10)

Average number of restaurants in every area with radius=300m: 22.8


,Neighborhood,Latitude,Longitude,Distance,X,Y,Restaurants in area,Distance to cafe
0,Grand Boulevard,41.816814,-87.606708,2.211630e+07,-5.389478e+06,1.152288e+07,8,88.692431
1,Printers Row,41.874371,-87.627607,2.211316e+07,-5.379867e+06,1.152233e+07,59,909.465834
2,United Center,41.888852,-87.667069,2.211578e+07,-5.376142e+06,1.152609e+07,51,938.323448
3,Sheffield & DePaul,41.921661,-87.658335,2.211193e+07,-5.371461e+06,1.152342e+07,55,103.370888
4,Humboldt Park,41.887823,-87.740596,2.212319e+07,-5.373476e+06,1.153454e+07,4,2909.053530
5,Garfield Park,41.888185,-87.695400,2.211866e+07,-5.375158e+06,1.152937e+07,17,1432.966189
6,North Lawndale,41.869869,-87.720239,2.212280e+07,-5.377003e+06,1.153314e+07,7,907.699344
7,Little Village,41.834801,-87.687399,2.212272e+07,-5.383628e+06,1.153119e+07,6,5000.000000
8,Armour Square,41.847127,-87.629201,2.211580e+07,-5.383975e+06,1.152390e+07,43,1967.812453
9,Avalon Park,41.751502,-87.585655,2.212014e+07,-5.400296e+06,1.152381e+07,5,5000.000000


Now let's calculate the **distance to nearest cafe from every area candidate center** (not only those within 300m - we want distance to closest one, regardless of how distant it is).

In [56]:
distances_to_cafe = []

for ax, ay in zip(neighborhoods.X, neighborhoods.Y):
    min_distance = 5000
    for res in cafe_places.values():
        cx = res[7]
        cy = res[8]
        d = calc_xy_distance(ax, ay, cx, cy)
        if d<min_distance:
            min_distance = d
    distances_to_cafe.append(min_distance)

neighborhoods['Distance to cafe'] = distances_to_cafe

In [57]:
neighborhoods.head(10)

,Neighborhood,Latitude,Longitude,Distance,X,Y,Restaurants in area,Distance to cafe
0,Grand Boulevard,41.816814,-87.606708,2.211630e+07,-5.389478e+06,1.152288e+07,8,88.692431
1,Printers Row,41.874371,-87.627607,2.211316e+07,-5.379867e+06,1.152233e+07,59,909.465834
2,United Center,41.888852,-87.667069,2.211578e+07,-5.376142e+06,1.152609e+07,51,938.323448
3,Sheffield & DePaul,41.921661,-87.658335,2.211193e+07,-5.371461e+06,1.152342e+07,55,103.370888
4,Humboldt Park,41.887823,-87.740596,2.212319e+07,-5.373476e+06,1.153454e+07,4,2909.053530
5,Garfield Park,41.888185,-87.695400,2.211866e+07,-5.375158e+06,1.152937e+07,17,1432.966189
6,North Lawndale,41.869869,-87.720239,2.212280e+07,-5.377003e+06,1.153314e+07,7,907.699344
7,Little Village,41.834801,-87.687399,2.212272e+07,-5.383628e+06,1.153119e+07,6,5000.000000
8,Armour Square,41.847127,-87.629201,2.211580e+07,-5.383975e+06,1.152390e+07,43,1967.812453
9,Avalon Park,41.751502,-87.585655,2.212014e+07,-5.400296e+06,1.152381e+07,5,5000.000000


In [58]:
print('Average distance to closest cafe from each area center:', neighborhoods['Distance to cafe'].mean())

Average distance to closest cafe from each area center: 1888.1874253043272


OK, so **on average cafe can be found within ~2km** from every area center candidate. So we need to filter our areas carefully!

Let's create a map showing **heatmap / density of restaurants** and try to extract some meaningful insights from that. Also, let's show few circles indicating distance of 1km, 2km and 3km from Chicago center.

In [59]:
restaurant_latlons = [[res[2], res[3]] for res in restaurants.values()]

cafe_latlons = [[res[2], res[3]] for res in cafe_places.values()]

In [60]:
from folium import plugins
from folium.plugins import HeatMap

def boroughs_style(feature):
    return { 'color': 'blue', 'fill': False }

map_chicago = folium.Map(location=chicago, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_chicago) #cartodbpositron cartodbdark_matter
HeatMap(restaurant_latlons).add_to(map_chicago)
folium.Marker(chicago).add_to(map_chicago)
folium.Circle(chicago, radius=1000, fill=False, color='white').add_to(map_chicago)
folium.Circle(chicago, radius=2000, fill=False, color='white').add_to(map_chicago)
folium.Circle(chicago, radius=3000, fill=False, color='white').add_to(map_chicago)

map_chicago

Looks like a few pockets of low restaurant density closest to city center can be found **north and south from Chicago center**. 

Let's create another heatmap map showing **heatmap/density of cafes** only.

In [25]:
map_chicago = folium.Map(location=chicago, zoom_start=13)
folium.TileLayer('cartodbpositron').add_to(map_chicago) #cartodbpositron cartodbdark_matter
HeatMap(cafe_latlons).add_to(map_chicago)
folium.Marker(chicago).add_to(map_chicago)
folium.Circle(chicago, radius=1000, fill=False, color='white').add_to(map_chicago)
folium.Circle(chicago, radius=2000, fill=False, color='white').add_to(map_chicago)
folium.Circle(chicago, radius=3000, fill=False, color='white').add_to(map_chicago)

map_chicago

This map indicates higher density of existing cafes directly north and west from Chicago center, with closest pockets of **low cafe density positioned north, south-west and south from city center**.

Based on this we will now focus our analysis on areas *north, south-west and south from Chicago center* - we will move the center of our area of interest and reduce it's size to have a radius of **2.5km**. 

In [26]:
roi_x_min = -5.379867e+06  
roi_y_max = 1.152233e+07
roi_width = 5000
roi_height = 5000
roi_center_x = roi_x_min 
roi_center_y = roi_y_max 
roi_center_lon, roi_center_lat = xy_to_lonlat(roi_center_x, roi_center_y)
roi_center = [roi_center_lat, roi_center_lon]
folium.TileLayer('cartodbpositron').add_to(map_chicago)
map_chicago = folium.Map(location=roi_center, zoom_start=14)
HeatMap(restaurant_latlons).add_to(map_chicago)
folium.Marker(chicago).add_to(map_chicago)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_chicago)

map_chicago

Not bad - this nicely covers all the pockets of low restaurant density closest to Chicago center.

Let's also create new, more dense grid of location candidates restricted to our new region of interest (let's make our location candidates 500m apart).

In [41]:
k = math.sqrt(3) / 2 # Vertical offset for hexagonal grid cells
x_step = 500
y_step = 500 * k 
roi_y_min = roi_center_y - 2500

roi_latitudes = []
roi_longitudes = []
roi_xs = []
roi_ys = []
for i in range(0, int(51/k)):
    y = roi_y_min + i * y_step
    x_offset = 50 if i%2==0 else 0
    for j in range(0, 51):
        x = roi_x_min + j * x_step + x_offset
        d = calc_xy_distance(roi_center_x, roi_center_y, x, y)
        if (d <= 2501):
            lon, lat = xy_to_lonlat(x, y)
            roi_latitudes.append(lat)
            roi_longitudes.append(lon)
            roi_xs.append(x)
            roi_ys.append(y)

print(len(roi_latitudes), 'candidate neighborhood centers generated.')

50 candidate neighborhood centers generated.


Now let's calculate two most important things for each location candidate: **number of restaurants in vicinity** (we'll use radius of **250 meters**) and **distance to closest cafe**.

In [42]:
def count_restaurants_nearby(x, y, restaurants, radius=1000):    
    count = 0
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=radius:
            count += 1
    return count

def find_nearest_restaurant(x, y, restaurants):
    d_min = 7000
    for res in restaurants.values():
        res_x = res[7]; res_y = res[8]
        d = calc_xy_distance(x, y, res_x, res_y)
        if d<=d_min:
            d_min = d
    return d_min

roi_restaurant_counts = []
roi_cafe_distances = []

print('Generating data on location candidates... ', end='')
for x, y in zip(roi_xs, roi_ys):
    count = count_restaurants_nearby(x, y, restaurants, radius=250)
    roi_restaurant_counts.append(count)
    distance = find_nearest_restaurant(x, y, cafe_places)
    roi_cafe_distances.append(distance)
print('done.')


Generating data on location candidates... done.


In [43]:
# Let's put this into dataframe
df_roi_locations = pd.DataFrame({'Latitude':roi_latitudes,
                                 'Longitude':roi_longitudes,
                                 'X':roi_xs,
                                 'Y':roi_ys,
                                 'restaurants nearby':roi_restaurant_counts,
                                 'Distance to cafe':roi_cafe_distances})

df_roi_locations.head(10)

,Latitude,Longitude,X,Y,restaurants nearby,Distance to cafe
0,41.879565,-87.608104,-5379817.0,1.151983e+07,0,1657.186705
1,41.878421,-87.611380,-5379867.0,1.152026e+07,0,1276.804478
2,41.881361,-87.612693,-5379367.0,1.152026e+07,0,1139.437946
3,41.884301,-87.614006,-5378867.0,1.152026e+07,0,1209.680827
4,41.877865,-87.614918,-5379817.0,1.152070e+07,0,881.935036
5,41.880805,-87.616231,-5379317.0,1.152070e+07,0,704.200657
6,41.883745,-87.617545,-5378817.0,1.152070e+07,0,831.724512
7,41.886685,-87.618858,-5378317.0,1.152070e+07,0,1165.633552
8,41.876721,-87.618193,-5379867.0,1.152113e+07,0,641.694094
9,41.879661,-87.619506,-5379367.0,1.152113e+07,0,282.595942


OK. Let us now **filter** those locations: we're interested only in **locations with no more than one restaurant in radius of 250 meters**, and **no cafes in radius of 400 meters**.

In [44]:
good_res_count = np.array((df_roi_locations['restaurants nearby']<=1))
print('Locations with no restaurants nearby:', good_res_count.sum())

good_caf_distance = np.array(df_roi_locations['Distance to cafe']>=400)
print('Locations with no cafes within 400m:', good_caf_distance.sum())

good_locations = np.logical_and(good_res_count, good_caf_distance)
print('Locations with both conditions met:', good_locations.sum())

df_good_locations = df_roi_locations[good_locations]


Locations with no restaurants nearby: 32
Locations with no cafes within 400m: 41
Locations with both conditions met: 28


Let's see how this looks on a map.

In [45]:
good_latitudes = df_good_locations['Latitude'].values
good_longitudes = df_good_locations['Longitude'].values

good_locations = [[lat, lon] for lat, lon in zip(good_latitudes, good_longitudes)]

map_chicago = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_chicago)
HeatMap(restaurant_latlons).add_to(map_chicago)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.6).add_to(map_chicago)
folium.Marker(chicago).add_to(map_chicago)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_chicago) 

map_chicago

Looking good. We now have a bunch of locations fairly close to Chicago center and we know that each of those locations has no more than one restaurant in radius of 250m, and no cafes closer than 400m. Any of those locations is a potential candidate for a new cafes, at least based on nearby competition.

Let's now show those good locations in a form of heatmap:

In [46]:
map_chicago = folium.Map(location=roi_center, zoom_start=14)
HeatMap(good_locations, radius=25).add_to(map_chicago)
folium.Marker(chicago).add_to(map_chicago)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_chicago)
map_chicago

Looking good. What we have now is a clear indication of zones with low number of restaurants in vicinity, and *no* cafess at all nearby.

Let us now **cluster** those locations to create **centers of zones containing good locations**. Those zones, their centers and addresses will be the final result of our analysis. 

In [47]:
from sklearn.cluster import KMeans

number_of_clusters = 3

good_xys = df_good_locations[['X', 'Y']].values
kmeans = KMeans(n_clusters=number_of_clusters, random_state=0).fit(good_xys)

cluster_centers = [xy_to_lonlat(cc[0], cc[1]) for cc in kmeans.cluster_centers_]

map_chicago = folium.Map(location=roi_center, zoom_start=14)
folium.TileLayer('cartodbpositron').add_to(map_chicago)
HeatMap(restaurant_latlons).add_to(map_chicago)
folium.Circle(roi_center, radius=2500, color='white', fill=True, fill_opacity=0.4).add_to(map_chicago)
folium.Marker(chicago).add_to(map_chicago)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=900, color='green', fill=True, fill_opacity=0.25).add_to(map_chicago) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_chicago)

map_chicago

Not bad - our clusters represent groupings of most of the candidate locations and cluster centers are placed nicely in the middle of the zones 'rich' with location candidates.

Addresses of those cluster centers will be a good starting point for exploring the neighborhoods to find the best possible location based on neighborhood specifics.

Let's see those zones on a city map without heatmap, using shaded areas to indicate our clusters:

In [48]:
map_chicago = folium.Map(location=roi_center, zoom_start=14)
folium.Marker(chicago).add_to(map_chicago)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#00000000', fill=True, fill_color='#0066ff', fill_opacity=0.07).add_to(map_chicago)
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.CircleMarker([lat, lon], radius=2, color='blue', fill=True, fill_color='blue', fill_opacity=1).add_to(map_chicago)
for lon, lat in cluster_centers:
    folium.Circle([lat, lon], radius=900, color='green', fill=False).add_to(map_chicago) 

map_chicago

In [53]:
candidate_area_addresses = []
print('==============================================================')
print('Lat-Long of centers of areas recommended for further analysis')
print('==============================================================\n')
for lon, lat in cluster_centers:
    geolocator = Nominatim(user_agent="ch_explorer")
    location = geolocator.reverse(lon, lat)
    candidate_area_addresses.append(location.address)    
    x, y = lonlat_to_xy(lon, lat)
    d = calc_xy_distance(x, y, X, Y)
    print('{},{} => {:f}m from Chicago center'.format(lat, lon, d/10000))

Lat-Long of centers of areas recommended for further analysis

41.87367302564292,-87.63954943427389 => 2211.441701m from Chicago center
41.88114797943391,-87.61638420659332 => 2211.142952m from Chicago center
41.885278274755876,-87.63300173893865 => 2211.271132m from Chicago center


This concludes our analysis. We have created 3 addresses representing centers of zones containing locations with low number of restaurants and no cafes nearby, all zones being fairly close to city center (all less than 2km from Chicago center). Although zones are shown on map with a radius of ~500 meters (green circles), their shape is actually very irregular and their centers/addresses should be considered only as a starting point for exploring area neighborhoods in search for potential restaurant locations.

In [50]:
map_chicago = folium.Map(location=roi_center, zoom_start=14)
folium.Circle(chicago, radius=50, color='red', fill=True, fill_color='red', fill_opacity=1).add_to(map_chicago)
for lonlat, addr in zip(cluster_centers, candidate_area_addresses):
    folium.Marker([lonlat[1], lonlat[0]], popup=addr).add_to(map_chicago) 
for lat, lon in zip(good_latitudes, good_longitudes):
    folium.Circle([lat, lon], radius=250, color='#0000ff00', fill=True, fill_color='#0066ff', fill_opacity=0.05).add_to(map_chicago)
map_chicago

## Results and Discussion <a name="results"></a>

Our analysis shows that although there is a great number of restaurants in Chicago, there are pockets of low restaurant density fairly close to city center. Highest concentration of restaurants was detected west from city center, so we focused our attention to areas north, east, south-west and south.

After directing our attention to the narrow area of interest we first created a dense grid of location candidates (spaced 500m apart); those locations were then filtered so that those with more than one restaurants in radius of 250m and those with an Italian restaurant closer than 400m were removed.

Those location candidates were then clustered to create zones of interest which contain greatest number of location candidates. Latitude and longitudes of centers of those zones were identified, which can be used as markers/starting points for more detailed local analysis based on other factors.

Result of all this is 3 zones containing largest number of potential new cafe locations based on number of and distance to existing venues - both restaurants in general and cafes particularly. This, of course, does not imply that those zones are actually optimal locations for a new cafe! Purpose of this analysis was to only provide info on areas close to Chicago center but not crowded with existing restaurants (particularly cafes) - it is entirely possible that there is a very good reason for small number of restaurants in any of those areas, reasons which would make them unsuitable for a new restaurant regardless of lack of competition in the area. Recommended zones should therefore be considered only as a starting point for more detailed analysis which could eventually result in location which has not only no nearby competition but also other factors taken into account and all other relevant conditions met.

## Conclusion <a name="conclusion"></a>

Purpose of this project was to identify areas close to Chicago center with low number of restaurants (particularly cafes) in order to aid stakeholders in narrowing down the search for optimal location for a new cafe. By calculating restaurant density distribution from Foursquare data we have first identified general neighborhood locations that justify further analysis, and then generated extensive collection of locations which satisfy some basic requirements regarding existing nearby restaurants. Clustering of those locations was then performed in order to create major zones of interest (containing greatest number of potential locations) and addresses of those zone centers were created to be used as starting points for final exploration by stakeholders.

Final decision on optimal restaurant location will be made by stakeholders based on specific characteristics of neighborhoods and locations in every recommended zone, taking into consideration additional factors like attractiveness of each location (proximity to park or water), levels of noise / proximity to major roads, real estate availability, prices, social and economic dynamics of every neighborhood etc.